# Alhazen Toolkit   

> A set of Langchain tools that populate and query a CEIFNS database by (1) Build collections of expressions; (2) locate and load items that represent expressions; (3) segregate the parts of items as 'fragments'; (4) analyze the fragments to generate notes that can then be summarized to provide summaries. 

In [ ]:
#| default_exp toolkit

# Base Toolkit for the 

In [ ]:
#| hide
from nbdev import *

In [ ]:
#| export

import local_resources.linkml as linkml

from alhazen.core import OllamaRunner
from alhazen.tools.basic import *
from alhazen.tools.metadata_extraction_tool import * 
from alhazen.tools.protocol_extraction_tool import * 
from alhazen.tools.paperqa_emulation_tool import * 
from alhazen.tools.tiab_classifier_tool import * 
from alhazen.tools.tiab_extraction_tool import * 
from alhazen.tools.tiab_mapping_tool import * 
from alhazen.utils.ceifns_db import *

from langchain.chat_models.base import BaseChatModel
from langchain.agents.agent import RunnableAgent
from langchain.tools import BaseTool
from langchain.pydantic_v1 import BaseModel, Extra, Field, root_validator
from langchain.schema.prompt_template import format_document

from importlib_resources import files
import local_resources.prompt_elements as prompt_elements

from sqlalchemy.orm import sessionmaker
from typing import List, Any, Optional

# Primary Agent Toolkit

In [ ]:
#| export
# NOTE - Use LangChain's SQL_DATABASE TOOLKIT AS A MODEL 
# https://github.com/langchain-ai/langchain/blob/535db72607c4ae308566ede4af65295967bb33a8/libs/community/langchain_community/agent_toolkits/sql/toolkit.py#L18
#
# Use environment variables to denote the database name + base file location
# os.environ['ALHAZEN_DB_NAME'] = 'em_tech'
# os.environ['LOCAL_FILE_PATH'] = '/Users/gburns/alhazen/'

class AlhazenToolkit(BaseModel):
    '''Toolkit for building and querying an Alhazen CEIFNS (pron. 'SAI-FiNS') database 
    (CEIFNS = Collection-Expression-Item-Fragment-Note-Summary).'''

    # The local literature database (Collections, Expressions, Items, and Fragments)
    db: Ceifns_LiteratureDb = Field(exclude=True)
    llm : BaseChatModel = Field(exclude=True)
    agent : Optional[RunnableAgent] = Field(exclude=True) 
    
    class Config:
        """Configuration for this pydantic object."""
        arbitrary_types_allowed = True

    def get_tools(self) -> List[BaseTool]:
        """Get the tools in the toolkit."""

        add_collection_tool_description = (
            "This tool builds a new collection of scientific papers based on querying external databases."
        )
        add_collection_tool = AddCollectionFromEPMCTool(db=self.db, description=add_collection_tool_description)

        add_authors_to_collection_tool_description = (
            "This tool adds author- and institute-level information to an existing collection of scientific papers."
        )
        add_authors_to_collection_tool = AddAuthorsToCollectionTool(db=self.db, description=add_authors_to_collection_tool_description)
        
        describe_collection_tool_description = (
            "This tool describes the contents of a collection in the database. "
        )
        describe_collection_tool = DescribeCollectionCompositionTool(db=self.db, description=describe_collection_tool_description)

        delete_collection_tool_description = (
            "This tool deletes a collection from the database."
        )
        delete_collection_tool = DeleteCollectionTool(db=self.db, description=delete_collection_tool_description)

        check_expression_tool_description = (
            "This tool checks if the database contains a paper. "
        )
        check_expression_tool = CheckExpressionTool(db=self.db, description=check_expression_tool_description)

        retrieve_full_text_for_a_collection_tool_description = (
            "This tool retrieves full text versions of all papers in a collection."
        )
        retrieve_full_text_for_a_collection_tool = RetrieveFullTextToolForACollection(db=self.db, description=retrieve_full_text_for_a_collection_tool_description)

        retrieve_full_text_tool_description = (
            "This tool retrieves the full text version for a single paper. This is specified by one parameter: paper_id, which MUST start with 'doi:'. "
        )
        retrieve_full_text_tool = RetrieveFullTextTool(db=self.db, description=retrieve_full_text_tool_description)

        metadata_extraction_tool_description = (
            "This tool extracts all metadata for a specified type of experiment from a single full text paper."
        )
        metadata_extraction_tool = MetadataExtraction_EverythingEverywhere_Tool(
            db=self.db, llm=self.llm, description=metadata_extraction_tool_description
        )

        methods_metadata_extraction_tool_description = (
            "This tool extracts all metadata for a specified type of experiment from the methods section of a single full text paper."
        )
        methods_metadata_extraction_tool = MetadataExtraction_MethodsSectionOnly_Tool(
            db=self.db, llm=self.llm, description=methods_metadata_extraction_tool_description
        )

        protocol_extraction_tool_description = (
            "This tool draws a flowchart of the protocol described in a single full text paper."
        )
        protocol_extraction_tool = ProcotolEntitiesExtractionTool(
            db=self.db, llm=self.llm, description=protocol_extraction_tool_description
        )

        simple_extraction_tool_description = (
            "This tool extracts information from a single full text paper based on a specific question."
        )
        simple_extraction_tool = SimpleExtractionWithRAGTool(
            db=self.db, llm=self.llm, description=simple_extraction_tool_description
        )

        paperqa_emulation_tool_description = (
            "This tool writes a short essay to answer a scientific question."
        )
        paperqa_emulation_tool = PaperQAEmulationTool(
            db=self.db, llm=self.llm, description=paperqa_emulation_tool_description
        )

        tiab_classifier_tool_description = (
            "This tool classifies a paper as a trial, intervention, analysis, or background paper."
        )
        tiab_classifier_tool = TitleAbstractClassifier_OneDocAtATime_Tool(
            db=self.db, llm=self.llm, description=tiab_classifier_tool_description
        )

        tiab_mapper_tool_description = (
            "This analyzes the title and abstract to provide descriptions of background information, the goals and methods, and the results and conclusions for the study."
        )
        tiab_mapper_tool = TitleAbstractDiscourseMappingTool(
            db=self.db, llm=self.llm, description=tiab_mapper_tool_description
        )

        tiab_extract_tool_description = (
            "This analyzes the title and abstract to provide descriptions of background information, the goals and methods, and the results and conclusions for the study."
        )
        tiab_extract_tool = TitleAbstractExtraction_OneDocAtATime_Tool(
            db=self.db, llm=self.llm, description=tiab_extract_tool_description
        )

        tool_list = [
            add_collection_tool,
            add_authors_to_collection_tool,
            describe_collection_tool,
            delete_collection_tool,
            retrieve_full_text_tool,
            retrieve_full_text_for_a_collection_tool,
            metadata_extraction_tool,
            methods_metadata_extraction_tool,
            simple_extraction_tool,
            paperqa_emulation_tool,
            protocol_extraction_tool,
            check_expression_tool,
            tiab_classifier_tool,
            tiab_mapper_tool,
            tiab_extract_tool
        ]

        #trial_run_tool_description = (
        #    "This tool executes my planning chain for a given user instruction and reports back on what I plan do."
        #)
        #trial_run_tool = IntrospectionTool(description=trial_run_tool_description)
        #tool_list.append(trial_run_tool)

        return tool_list
    

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        if self.db.session is None:
            session_class = sessionmaker(bind=self.db.engine)
            self.db.session = session_class()

# Metadata Extraction Toolkit for Experimentation 

In [ ]:
#| export

class MetadataExtractionToolkit(BaseModel):
    '''Toolkit for running and testing Alhazen Metadata Extraction Tools'''

    # The local literature database (Collections, Expressions, Items, and Fragments)
    db: Ceifns_LiteratureDb = Field(exclude=True)
    llm : BaseChatModel = Field(exclude=True)
    agent : Optional[RunnableAgent] = Field(exclude=True) 
    
    class Config:
        """Configuration for this pydantic object."""
        arbitrary_types_allowed = True

    def get_tools(self) -> List[BaseTool]:
        """Get the tools in the toolkit."""

        metadata_extraction_everything_everywhere_tool_description = (
            "This tool asks all metadata questions in a single shot over the whole document."
        )
        metadata_extraction_everything_everywhere_tool = MetadataExtraction_EverythingEverywhere_Tool(
            db=self.db, llm=self.llm, description=metadata_extraction_everything_everywhere_tool_description
        )

        metadata_extraction_MethodsSectionOnly_tool_description = (
            "This tool asks all metadata questions in a single shot over the methods section only."
        )
        metadata_extraction_MethodsSectionOnly_tool = MetadataExtraction_MethodsSectionOnly_Tool(
            db=self.db, llm=self.llm, description=metadata_extraction_MethodsSectionOnly_tool_description
        )

        metadata_extraction_rag_on_sections_tool_description = (
            "This tool asks each question separately over whole sections selected by RAG."
        )
        metadata_extraction_rag_on_sections_tool = MetadataExtraction_RAGOnSections_Tool(
            db=self.db, llm=self.llm, description=metadata_extraction_rag_on_sections_tool_description
        )

        metadata_extraction_simple_extraction_tool_description = (
            "This tool asks a single question over sections selected by RAG."
        )
        metadata_extraction_simple_extraction_tool = SimpleExtractionWithRAGTool(
            db=self.db, llm=self.llm, description=metadata_extraction_simple_extraction_tool_description
        )

        tool_list = [
            metadata_extraction_everything_everywhere_tool,
            metadata_extraction_MethodsSectionOnly_tool,
            metadata_extraction_rag_on_sections_tool,
            metadata_extraction_simple_extraction_tool
        ]

        return tool_list
    

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        if self.db.session is None:
            session_class = sessionmaker(bind=self.db.engine)
            self.db.session = session_class()